# Домашнее задание 4

In [1]:
import numpy as np
import pandas as pd
import sklearn.linear_model as slmodel
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import f
from scipy import stats

In [2]:
table = pd.read_excel('RPL.xlsx', 'Лист1')
table.head()

,Место,Клуб,Матчи,Победы,Ничьи,Поражения,Забито,Пропущено,Разница,Очки
0,1,Спартак,30,16,10,4,51,30,21,58
1,2,Локомотив,30,16,9,5,53,22,31,57
2,3,Зенит,30,15,9,6,50,34,16,54
3,4,Торпедо,30,15,7,8,51,40,11,52
4,5,Крылья Советов,30,14,8,8,38,22,16,50


# Тест Зарембки

In [3]:
table['Очки1'] = table['Очки']/(np.exp(sum(np.log(table['Очки']))/len(table['Очки'])))

In [4]:
x = table[['Забито', 'Пропущено']].values
y1 = table.loc[:,'Очки1'].values
y2 = np.log(table.loc[:,'Очки1'].values)
model1 = slmodel.LinearRegression()
model1.fit(x, y1)
print(model1.intercept_, model1.coef_)
model2 = slmodel.LinearRegression()
model2.fit(x, y2)
print(model2.intercept_, model2.coef_)

0.6127394449379584 [ 0.02234613 -0.01030976]
-0.29505590733963316 [ 0.02074965 -0.01274928]


In [5]:
RSS1 = np.sum((table['Очки1']- model1.coef_[0]*table['Забито']\
               - model1.coef_[1]*table['Пропущено'] - model1.intercept_)**2)
RSS2 = np.sum((table['Очки1']- model2.coef_[0]*table['Забито']\
               - model2.coef_[1]*table['Пропущено'] - model2.intercept_)**2)
print(RSS1, RSS2)

4.647323987916629 344.23989247070494


In [6]:
chi2 = len(table['Очки'])/2 * abs(np.log(RSS1/RSS2))
print(chi2)

654.3671757207645


In [7]:
table['Очки1'].min()

0.2527228661416779

# Бокс-Кокс

In [8]:
step = 0.01
i = 0
eps = 10**(-12)
result = np.zeros(100)
table['Забито1'] = table['Забито'] + 2
table['Пропущено1'] = table['Пропущено'] + 2
table['Очки2'] = table['Очки1'] + 2
while i <= 1:
    table['Очки3'] = stats.boxcox(table['Очки2'], lmbda= i)
    table['Забито3'] = stats.boxcox(table['Забито1'], lmbda= i)
    table['Пропущено3'] = stats.boxcox(table['Пропущено1'], lmbda= i)
    x = table[['Забито3', 'Пропущено3']].values
    y3 = table['Очки3']
    model3 = slmodel.LinearRegression()
    model3.fit(x, y3)
    result[int(i/step)] = np.sum((table['Очки3']- model3.coef_[0]*table['Забито3']\
               - model3.coef_[1]*table['Пропущено3'] - model3.intercept_)**2)
    i = i + step
print(np.argmin(result)*step)
    

0.14
